# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Vocabulary

---

# Observations et environnement
---

In [2]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [3]:
import importlib
import pandas as pd
import seaborn as sns

## Articles & Labels

In [4]:
import patat.db.article_db

In [5]:
article_db = patat.db.article_db.ArticleDb()

/Users/fm/Desktop/Work/Patat/patat/db/article_db.py:37: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [29]:
df_corpus = article_db._df.copy()

In [30]:
df_corpus['text'] = df_corpus['title'] + '\n' + df_corpus['article']

## Textes

In [9]:
df_infox = pd.read_csv('data/tmp/221219-InfoxTexts')

In [10]:
df_afp = pd.read_csv('data/tmp/221219-AfpTexts')

# Experience
---

### Preprocessor

In [11]:
import patat.util.text

preprocessor = patat.util.text.preprocess


### Tokenizer

In [12]:
from patat.ml.lex_analyser import LexAnalyser

lex = LexAnalyser()

tokenizer = lex.get_lemmas_from_text

### Vocabulary

In [13]:
df_texts = pd.concat([df_infox,df_afp])

In [20]:
df_texts = df_texts.reset_index()

In [21]:
texts = df_texts['text']

In [22]:
values = df_texts['infox'].astype(int)

In [23]:
df_words = lex.get_df_words(texts, values, preprocessor, tokenizer)

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
df_words

,count_0,count_1,count
de,115224,24646,139870
la,63847,13219,77066
le,49789,10182,59971
et,39056,10379,49435
les,38357,10599,48956
...,...,...,...
pharmacol,0,3,3
pharmacologique,0,6,6
pharmacologistes,0,1,1
pharmacologue,0,1,1


### Rare Words

In [25]:
for occ_rare in [1,2,3,4,5,6,7]:
    df_words['rare_'+str(occ_rare)]=(df_words['count_0']<=occ_rare)&(df_words['count_1']<=occ_rare)

In [26]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7
de,115224,24646,139870,False,False,False,False,False,False,False
la,63847,13219,77066,False,False,False,False,False,False,False
le,49789,10182,59971,False,False,False,False,False,False,False
et,39056,10379,49435,False,False,False,False,False,False,False
les,38357,10599,48956,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
pharmacol,0,3,3,False,False,True,True,True,True,True
pharmacologique,0,6,6,False,False,False,False,False,True,True
pharmacologistes,0,1,1,True,True,True,True,True,True,True
pharmacologue,0,1,1,True,True,True,True,True,True,True


### Common words

In [27]:
for common_size in [50,80,120,200,500]:
    common_0 = df_words.sort_values('count_0',ascending=False).head(common_size).index
    common_1 = df_words.sort_values('count_1',ascending=False).head(common_size).index
    df_words['common_'+str(common_size)]=(df_words.index.isin(common_0))&(df_words.index.isin(common_1))

In [28]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7,common_50,common_80,common_120,common_200,common_500
de,115224,24646,139870,False,False,False,False,False,False,False,True,True,True,True,True
la,63847,13219,77066,False,False,False,False,False,False,False,True,True,True,True,True
le,49789,10182,59971,False,False,False,False,False,False,False,True,True,True,True,True
et,39056,10379,49435,False,False,False,False,False,False,False,True,True,True,True,True
les,38357,10599,48956,False,False,False,False,False,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pharmacol,0,3,3,False,False,True,True,True,True,True,False,False,False,False,False
pharmacologique,0,6,6,False,False,False,False,False,True,True,False,False,False,False,False
pharmacologistes,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
pharmacologue,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False


### Vocab & Ignore words

In [31]:
ignore_words_small = list(df_words[df_words['rare_1']].index) + list(df_words[df_words['common_80']].index)

In [32]:
ignore_words_medium = list(df_words[df_words['rare_2']].index) + list(df_words[df_words['common_80']].index)

In [33]:
ignore_words_large = list(df_words[df_words['rare_5']].index) + list(df_words[df_words['common_200']].index)

## Topoi

In [38]:
topos = 'officine'

In [39]:
df_corpus[df_corpus['text'].str.contains(topos,case=False)]

,url,title,article,author,tag,abstract,site,date_iso,url_h,c_count,text
13,https://www.francesoir.fr/societe/onde-de-choc...,Onde de choc au sein des réseaux du fact-check...,"ENQUÊTE - Depuis plusieurs mois, FranceSoir en...",Auteur(s)\nFranceSoir,[],NaN,www.francesoir.fr,2022-11-28T06:30:00,64992bb518b0bbec3b45df023a0341b114132f83,NaN,Onde de choc au sein des réseaux du fact-check...
108,https://ripostelaique.com/de-linjection-pro-sp...,De l’injection pro-spike à l’injonction anti-P...,Les preuves s’amoncellent sur l’inutilité d’un...,Lucien Samir Oulahbib,"['NOUVEL ORDRE MONDIAL, RIPOSTE LAIQUE']",NaN,ripostelaique.com,2022-10-10T00:00:00,3d4368915518ad433bfe06112f9518cc04af380e,4082.0,De l’injection pro-spike à l’injonction anti-P...
110,https://ripostelaique.com/lea-salame-na-t-elle...,Léa Salamé n’a-t-elle pas honte d’oser se prét...,"Fourberie contre loyauté, mensonge contre sinc...",Josiane Filio,['MEDIAS'],NaN,ripostelaique.com,2022-10-27T00:00:00,5ab97f4d644694a2f22d1d53888de4468cc5c096,6981.0,Léa Salamé n’a-t-elle pas honte d’oser se prét...
146,https://ripostelaique.com/lamia-el-aaraje-port...,"Lamia El Aaraje, porte-parole du PS, insulte c...","Lamia El Aaraje, porte-parole du PS, ce qui en...",Sophie Durand,['MEDIAS'],NaN,ripostelaique.com,2022-09-29T00:00:00,d143a21cb0540a72c73fa5d9180131575a437e8b,6229.0,"Lamia El Aaraje, porte-parole du PS, insulte c..."
193,https://ripostelaique.com/reactions-au-discour...,Réactions au discours de Poutine : misère de l...,Au-delà des vicissitudes concernant ce qui se ...,Lucien Samir Oulahbib,['MEDIAS'],NaN,ripostelaique.com,2022-10-02T00:00:00,8b70798cd259307e6b326495f15b37c1549b5a37,7585.0,Réactions au discours de Poutine : misère de l...
...,...,...,...,...,...,...,...,...,...,...,...
84982,https://www.lelibrepenseur.org/covidisme-le-mo...,Covidisme : le Molnupiravir a été écarté par l...,Encore une connerie réalisée par Véran dans l’...,Par\nAdmin,NaN,NaN,www.lelibrepenseur.org,2021-12-10T00:00:00,ec303bc7678b96e543041f2b92c21a9985f01001,2380.0,Covidisme : le Molnupiravir a été écarté par l...
85248,https://www.lelibrepenseur.org/business-covid-...,"Business Covid : une manne de 1,6 milliard pou...",C’est certainement une des raisons qui expliqu...,Par\nAguellid,NaN,NaN,www.lelibrepenseur.org,2021-10-16T00:00:00,4793840d1807a1dd0a31dbab2af66eb0013e9344,1459.0,"Business Covid : une manne de 1,6 milliard pou..."
85927,https://www.lelibrepenseur.org/covidisme-et-ma...,Covidisme et maltraitance des enfants à l'école !,Vidéo d’une extrême violence qui démontre à qu...,Par\nAdmin,NaN,NaN,www.lelibrepenseur.org,2021-06-11T00:00:00,c453f55c2ff44c0f9c6c16cca6ed98e235c7b636,537.0,Covidisme et maltraitance des enfants à l'écol...
86488,https://www.lelibrepenseur.org/libe-et-les-med...,"Libé et les médias, font-ils encore de l’info ...",Article fort intéressant concernant le comport...,Par\nAguellid,NaN,NaN,www.lelibrepenseur.org,2020-08-03T00:00:00,10891b77e1165a4d6ed5a5835e73faf92240e52d,27238.0,"Libé et les médias, font-ils encore de l’info ..."


# Sauvegarde des résultats
---

In [ ]:
df_words.to_csv('data/prod/221125-InfoxWords.csv')

# Conclusions
---

- Vocabulaire infox OK

# Bricolages
---

In [ ]:
df_texts

In [ ]:
df_texts=df_texts.reset_index()

In [ ]:
texts = df_texts['text']

In [ ]:
values = df_texts['infox']

In [ ]:
vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer,token_pattern='')
count_matrix = vectorizer.fit_transform(texts)

In [ ]:
count_array = count_matrix.toarray()
df_count = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())

In [ ]:
df_words = pd.DataFrame()
for value in values.unique():
    df_words['count_'+str(value)] = pd.DataFrame(df_count[values==value].sum(),columns=['count_0']).sort_values('count_0',ascending=False)
df_words['count']=df_count.sum(axis=0)

df_words.shape

In [ ]:
df_words

In [ ]:
importlib.reload(patat.ml.lex_analyser)

In [ ]:
lex = patat.ml.lex_analyser.LexAnalyser()

In [ ]:
lex.get_df_words(texts, values, preprocessor, tokenizer)

In [ ]:
import typing

In [ ]:
typing.reveal_type(tokenizer)

In [ ]:
values.unique()

In [ ]:
pd.MultiIndex.from_product([['value_count'],[0,1]])

In [1]:
from bs4 import BeautifulSoup

In [5]:
import requests

In [6]:
url = 'https://www.sudradio.fr/societe/sante-macron-annonce-un-plan-pour-sortir-dune-crise-sans-fin-a-lhopital-et-en-ville'

In [8]:
r = requests.get(url)

In [9]:
r

<Response [200]>

In [12]:
html = r.text

In [13]:
soup = BeautifulSoup(html)

In [24]:
soup.select('time')[0].

'\n                                                vendredi 6 janvier 2023 à 13:43                                            '